In [264]:
import pandas as pd
import numpy as np
# import scipy.stat as sps
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import ExcelWriter

In [265]:
%%time
df = pd.read_excel('card transactions.xlsx')

Wall time: 9.97 s


In [266]:
# summary
numrecords = len(df)
stat_df = pd.DataFrame(df.isnull().sum(axis = 0))
stat_df = stat_df.reset_index().rename(columns = {'index':'variable', 0 :"# NaN"})
stat_df["% populated"] = (1 - stat_df["# NaN"]/numrecords)*100
stat_df["# unique"] = ''

for i in range(len(list(df))):
    stat_df["# unique"][i] = df[list(df)[i]].nunique()
    
stat_df

C:\Users\beizh\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,variable,# NaN,% populated,# unique
0,Recnum,0,100.000000,96753
1,Cardnum,0,100.000000,1645
2,Date,0,100.000000,365
3,Merchnum,3375,96.511736,13091
4,Merch description,0,100.000000,13126
5,Merch state,1195,98.764896,227
6,Merch zip,4656,95.187746,4567
7,Transtype,0,100.000000,4
8,Amount,0,100.000000,34909
9,Fraud,0,100.000000,2


In [267]:
# df.head(10)

In [268]:
##Identify any exclusions, bad records

# Remove single large transaction outlier in Amount
df = df[df['Amount']<3000000]

# Remove all but P transaction type
df = df[df['Transtype'] == 'P']

len(df)

96397

In [269]:
# --------------- Fill in NAs ----------

### Merchnum 
# Transform 0 values into NA
df.loc[df['Merchnum'] == '0', 'Merchnum'] = np.NaN

# fill most frequenct merchnum using merch description
dict1 = df.set_index('Merch description')['Merchnum'].to_dict()
df['Merchnum'].fillna(df['Merch description'].map(dict1), inplace = True)

# fill most frequenct merchnum in this zip
dict1 = dict(zip(df['Merch zip'].dropna(), df['Merchnum']))
df['Merchnum'].fillna(df['Merch zip'].map(dict1), inplace = True)

pd.isnull(df['Merchnum']).sum()


1414

In [270]:
### Merch state and zip
numrecords - (df['Merch state'].isna() == df['Merch zip'].isna()).sum()


3810

In [271]:
### Merch zip
# df.groupby(['Merch description','Merch zip']).size()

# fill NA in Merch zip using Merch description
dict1 = df.set_index('Merch description')['Merch zip'].to_dict()
df['Merch zip'].fillna(df['Merch description'].map(dict1), inplace = True)

# fill NA in Merch zip using Merchnum
dict1 = df.set_index('Merchnum')['Merch zip'].to_dict()
df['Merch zip'].fillna(df['Merchnum'].map(dict1), inplace = True)

# fill NA in Merch zip using Cardnum
dict1 = df.set_index('Cardnum')['Merch zip'].to_dict()
df['Merch zip'].fillna(df['Cardnum'].map(dict1), inplace = True)
# It will automatically choose zip with highest frequency

pd.isnull(df['Merch zip']).sum()


543

In [272]:
dic = dict(zip(df['Cardnum'], df['Merch zip']))
dic[5142310525]

60108.0

In [273]:
### Merch state
# df.groupby(['Merch description','Merch state']).size()
# Each Merch description has one matched state

# fill NA in Merch state using Merch description
dict1 = df.set_index('Merch description')['Merch state'].to_dict()
df['Merch state'].fillna(df['Merch description'].map(dict1), inplace = True)

# fill NA in Merch state using Merchnum
dict1 = df.set_index('Merchnum')['Merch state'].to_dict()
df['Merch state'].fillna(df['Merchnum'].map(dict1), inplace = True)

# fill NA in Merch state using Merch zip

dict1 = dict(zip(df['Merch zip'].dropna(), df['Merch state']))
df['Merch state'].fillna(df['Merch zip'].map(dict1), inplace = True)

# fill NA in Merch state using Cardnum
dict1 = df.set_index('Cardnum')['Merch state'].to_dict()
df['Merch state'].fillna(df['Cardnum'].map(dict1), inplace = True)

pd.isnull(df['Merch state']).sum()


122

In [274]:
# Drop remained NA in three variables
df = df.dropna()
len(df)

94631

In [275]:
# df.groupby(['Cardnum','Date'])['Amount'].value_counts()
# df[df['Cardnum'] == 5142190439]

Cardnum     Date        Amount 
5142110081  2010-11-26  636.20     1
            2010-12-27  495.90     1
5142110402  2010-01-19  506.50     1
            2010-01-31  20.81      1
            2010-02-03  30.26      1
            2010-02-04  54.00      1
                        437.76     1
5142110434  2010-10-06  1551.64    1
5142110749  2010-04-19  62.94      1
            2010-06-13  794.85     1
5142110909  2010-01-17  351.24     1
            2010-01-20  64.40      1
            2010-02-09  301.79     1
            2010-03-21  128.00     1
            2010-03-22  90.99      1
                        766.88     1
            2010-04-05  435.47     1
            2010-04-12  30.40      1
                        161.25     1
            2010-04-13  809.45     1
            2010-05-18  79.80      2
            2010-06-12  288.00     1
            2010-07-07  1535.40    1
            2010-07-12  51.75      1
            2010-12-20  107.76     1
            2010-12-27  305.25     1
      

In [276]:
# --------------- Create variables ----------
### Amount expert variables

# Cardnum
# sum the amount at the same day first and then rolling
df2 = df.groupby(['Cardnum','Date'])['Amount'].sum().to_frame().reset_index().set_index('Date')
new_df = df.join(df.groupby(['Cardnum','Date'])['Amount'].sum(),on = ['Cardnum','Date'], rsuffix = '_cn_actual')



In [277]:
# (closed = 'left') in rolling function means when calculating past n days we exlude today's transactions

def amount_mean_func(column, window, rsuffix):
    dataframe = pd.DataFrame()
    dataframe = new_df.join(df2.groupby([column])['Amount'].rolling(window = window, closed = 'left').mean(), on = [column,'Date'], rsuffix = rsuffix)
    return dataframe

# def amount_max_func(column, window, rsuffix):
#     dataframe = pd.DataFrame()
#     dataframe = new_df.join(df2.groupby([column])['Amount'].rolling(window = window).max(), on = [column,'Date'], rsuffix = rsuffix)
#     return dataframe

# def amount_median_func(column, window, rsuffix):
#     dataframe = pd.DataFrame()
#     dataframe = new_df.join(df2.groupby([column])['Amount'].rolling(window = window).median(), on = [column,'Date'], rsuffix = rsuffix)
#     return dataframe

def amount_sum_func(column, window, rsuffix):
    dataframe = pd.DataFrame()
    dataframe = new_df.join(df2.groupby([column])['Amount'].rolling(window = window, closed = 'left').sum(), on = [column,'Date'], rsuffix = rsuffix)
    return dataframe

In [278]:
df2 = df.groupby(['Cardnum','Date'])['Amount'].sum().to_frame().reset_index().set_index('Date')
new_df = df.join(df.groupby(['Cardnum','Date'])['Amount'].sum(),on = ['Cardnum','Date'], rsuffix = '_cn_actual')

new_df = amount_mean_func(column = 'Cardnum',window = '1D',rsuffix = '_cn_1d_avg')
# new_df = amount_max_func(column = 'Cardnum',window = '1D',rsuffix = '_cn_1d_max')
# new_df = amount_median_func(column = 'Cardnum',window = '1D',rsuffix = '_cn_1d_median')
new_df = amount_sum_func(column = 'Cardnum',window = '1D',rsuffix = '_cn_1d_sum')
new_df['Amount_cn_1d_act-avg'] = new_df['Amount_cn_actual'] - new_df['Amount_cn_1d_avg']
# new_df['Amount_cn_1d_act-median'] = new_df['Amount_cn_actual'] - new_df['Amount_cn_1d_median']
new_df['Amount_cn_1d_act/avg'] = new_df['Amount_cn_actual']/new_df['Amount_cn_1d_avg']
# new_df['Amount_cn_1d_act/max'] = new_df['Amount_cn_actual']/new_df['Amount_cn_1d_max']
new_df['Amount_cn_1d_act/sum'] = new_df['Amount_cn_actual']/new_df['Amount_cn_1d_sum']
# new_df['Amount_cn_1d_act/median'] = new_df['Amount_cn_actual']/new_df['Amount_cn_1d_median']

new_df = amount_mean_func(column = 'Cardnum',window = '4D',rsuffix = '_cn_4d_avg')
new_df = amount_sum_func(column = 'Cardnum',window = '4D',rsuffix = '_cn_4d_sum')
new_df['Amount_cn_4d_act-avg'] = new_df['Amount_cn_actual'] - new_df['Amount_cn_4d_avg']
new_df['Amount_cn_4d_act/avg'] = new_df['Amount_cn_actual']/new_df['Amount_cn_4d_avg']
new_df['Amount_cn_4d_act/sum'] = new_df['Amount_cn_actual']/new_df['Amount_cn_4d_sum']

new_df = amount_mean_func(column = 'Cardnum',window = '7D',rsuffix = '_cn_7d_avg')
new_df = amount_sum_func(column = 'Cardnum',window = '7D',rsuffix = '_cn_7d_sum')
new_df['Amount_cn_7d_act-avg'] = new_df['Amount_cn_actual'] - new_df['Amount_cn_7d_avg']
new_df['Amount_cn_7d_act/avg'] = new_df['Amount_cn_actual']/new_df['Amount_cn_7d_avg']
new_df['Amount_cn_7d_act/sum'] = new_df['Amount_cn_actual']/new_df['Amount_cn_7d_sum']

new_df = amount_mean_func(column = 'Cardnum',window = '14D',rsuffix = '_cn_14d_avg')
new_df = amount_sum_func(column = 'Cardnum',window = '14D',rsuffix = '_cn_14d_sum')
new_df['Amount_cn_14d_act-avg'] = new_df['Amount_cn_actual'] - new_df['Amount_cn_14d_avg']
new_df['Amount_cn_14d_act/avg'] = new_df['Amount_cn_actual']/new_df['Amount_cn_14d_avg']
new_df['Amount_cn_14d_act/sum'] = new_df['Amount_cn_actual']/new_df['Amount_cn_14d_sum']

new_df = amount_mean_func(column = 'Cardnum',window = '30D',rsuffix = '_cn_30d_avg')
new_df = amount_sum_func(column = 'Cardnum',window = '30D',rsuffix = '_cn_30d_sum')
new_df['Amount_cn_30d_act-avg'] = new_df['Amount_cn_actual'] - new_df['Amount_cn_30d_avg']
new_df['Amount_cn_30d_act/avg'] = new_df['Amount_cn_actual']/new_df['Amount_cn_30d_avg']
new_df['Amount_cn_30d_act/sum'] = new_df['Amount_cn_actual']/new_df['Amount_cn_30d_sum']

In [279]:
df2 = df.groupby(['Merchnum','Date'])['Amount'].sum().to_frame().reset_index().set_index('Date')
new_df = new_df.join(df.groupby(['Merchnum','Date'])['Amount'].sum(),on = ['Merchnum','Date'], rsuffix = '_mn_actual')

new_df = amount_mean_func(column = 'Merchnum',window = '1D',rsuffix = '_mn_1d_avg')
new_df = amount_sum_func(column = 'Merchnum',window = '1D',rsuffix = '_mn_1d_sum')
new_df['Amount_mn_1d_act-avg'] = new_df['Amount_mn_actual'] - new_df['Amount_mn_1d_avg']
new_df['Amount_mn_1d_act/avg'] = new_df['Amount_mn_actual']/new_df['Amount_mn_1d_avg']
new_df['Amount_mn_1d_act/sum'] = new_df['Amount_mn_actual']/new_df['Amount_mn_1d_sum']

new_df = amount_mean_func(column = 'Merchnum',window = '4D',rsuffix = '_mn_4d_avg')
new_df = amount_sum_func(column = 'Merchnum',window = '4D',rsuffix = '_mn_4d_sum')
new_df['Amount_mn_4d_act-avg'] = new_df['Amount_mn_actual'] - new_df['Amount_mn_4d_avg']
new_df['Amount_mn_4d_act/avg'] = new_df['Amount_mn_actual']/new_df['Amount_mn_4d_avg']
new_df['Amount_mn_4d_act/sum'] = new_df['Amount_mn_actual']/new_df['Amount_mn_4d_sum']

new_df = amount_mean_func(column = 'Merchnum',window = '7D',rsuffix = '_mn_7d_avg')
new_df = amount_sum_func(column = 'Merchnum',window = '7D',rsuffix = '_mn_7d_sum')
new_df['Amount_mn_7d_act-avg'] = new_df['Amount_mn_actual'] - new_df['Amount_mn_7d_avg']
new_df['Amount_mn_7d_act/avg'] = new_df['Amount_mn_actual']/new_df['Amount_mn_7d_avg']
new_df['Amount_mn_7d_act/sum'] = new_df['Amount_mn_actual']/new_df['Amount_mn_7d_sum']

new_df = amount_mean_func(column = 'Merchnum',window = '14D',rsuffix = '_mn_14d_avg')
new_df = amount_sum_func(column = 'Merchnum',window = '14D',rsuffix = '_mn_14d_sum')
new_df['Amount_mn_14d_act-avg'] = new_df['Amount_mn_actual'] - new_df['Amount_mn_14d_avg']
new_df['Amount_mn_14d_act/avg'] = new_df['Amount_mn_actual']/new_df['Amount_mn_14d_avg']
new_df['Amount_mn_14d_act/sum'] = new_df['Amount_mn_actual']/new_df['Amount_mn_14d_sum']

new_df = amount_mean_func(column = 'Merchnum',window = '30D',rsuffix = '_mn_30d_avg')
new_df = amount_sum_func(column = 'Merchnum',window = '30D',rsuffix = '_mn_30d_sum')
new_df['Amount_mn_30d_act-avg'] = new_df['Amount_mn_actual'] - new_df['Amount_mn_30d_avg']
new_df['Amount_mn_30d_act/avg'] = new_df['Amount_mn_actual']/new_df['Amount_mn_30d_avg']
new_df['Amount_mn_30d_act/sum'] = new_df['Amount_mn_actual']/new_df['Amount_mn_30d_sum']

In [280]:
df2 = df.groupby(['Cardnum','Merchnum','Date'])['Amount'].sum().to_frame().reset_index().set_index('Date')
new_df = new_df.join(df.groupby(['Cardnum','Merchnum','Date'])['Amount'].sum(),on = ['Cardnum','Merchnum','Date'], rsuffix = '_cn_mn_actual')

new_df = new_df.join(df2.groupby(['Cardnum','Merchnum'])['Amount'].rolling(window = '1D', closed = 'left').mean(), on = ['Cardnum','Merchnum','Date'], rsuffix = '_cn_mn_1d_avg')
new_df = new_df.join(df2.groupby(['Cardnum','Merchnum'])['Amount'].rolling(window = '1D', closed = 'left').sum(), on = ['Cardnum','Merchnum','Date'], rsuffix = '_cn_mn_1d_sum')
new_df['Amount_cn_mn_1d_act-avg'] = new_df['Amount_cn_mn_actual'] - new_df['Amount_cn_mn_1d_avg']
new_df['Amount_cn_mn_1d_act/avg'] = new_df['Amount_cn_mn_actual']/new_df['Amount_cn_mn_1d_avg']
new_df['Amount_cn_mn_1d_act/sum'] = new_df['Amount_cn_mn_actual']/new_df['Amount_cn_mn_1d_sum']

new_df = new_df.join(df2.groupby(['Cardnum','Merchnum'])['Amount'].rolling(window = '4D', closed = 'left').mean(), on = ['Cardnum','Merchnum','Date'], rsuffix = '_cn_mn_4d_avg')
new_df = new_df.join(df2.groupby(['Cardnum','Merchnum'])['Amount'].rolling(window = '4D', closed = 'left').sum(), on = ['Cardnum','Merchnum','Date'], rsuffix = '_cn_mn_4d_sum')
new_df['Amount_cn_mn_4d_act-avg'] = new_df['Amount_cn_mn_actual'] - new_df['Amount_cn_mn_4d_avg']
new_df['Amount_cn_mn_4d_act/avg'] = new_df['Amount_cn_mn_actual']/new_df['Amount_cn_mn_4d_avg']
new_df['Amount_cn_mn_4d_act/sum'] = new_df['Amount_cn_mn_actual']/new_df['Amount_cn_mn_4d_sum']

new_df = new_df.join(df2.groupby(['Cardnum','Merchnum'])['Amount'].rolling(window = '7D', closed = 'left').mean(), on = ['Cardnum','Merchnum','Date'], rsuffix = '_cn_mn_7d_avg')
new_df = new_df.join(df2.groupby(['Cardnum','Merchnum'])['Amount'].rolling(window = '7D', closed = 'left').sum(), on = ['Cardnum','Merchnum','Date'], rsuffix = '_cn_mn_7d_sum')
new_df['Amount_cn_mn_7d_act-avg'] = new_df['Amount_cn_mn_actual'] - new_df['Amount_cn_mn_7d_avg']
new_df['Amount_cn_mn_7d_act/avg'] = new_df['Amount_cn_mn_actual']/new_df['Amount_cn_mn_7d_avg']
new_df['Amount_cn_mn_7d_act/sum'] = new_df['Amount_cn_mn_actual']/new_df['Amount_cn_mn_7d_sum']

new_df = new_df.join(df2.groupby(['Cardnum','Merchnum'])['Amount'].rolling(window = '14D', closed = 'left').mean(), on = ['Cardnum','Merchnum','Date'], rsuffix = '_cn_mn_14d_avg')
new_df = new_df.join(df2.groupby(['Cardnum','Merchnum'])['Amount'].rolling(window = '14D', closed = 'left').sum(), on = ['Cardnum','Merchnum','Date'], rsuffix = '_cn_mn_14d_sum')
new_df['Amount_cn_mn_14d_act-avg'] = new_df['Amount_cn_mn_actual'] - new_df['Amount_cn_mn_14d_avg']
new_df['Amount_cn_mn_14d_act/avg'] = new_df['Amount_cn_mn_actual']/new_df['Amount_cn_mn_14d_avg']
new_df['Amount_cn_mn_14d_act/sum'] = new_df['Amount_cn_mn_actual']/new_df['Amount_cn_mn_14d_sum']

new_df = new_df.join(df2.groupby(['Cardnum','Merchnum'])['Amount'].rolling(window = '30D', closed = 'left').mean(), on = ['Cardnum','Merchnum','Date'], rsuffix = '_cn_mn_30d_avg')
new_df = new_df.join(df2.groupby(['Cardnum','Merchnum'])['Amount'].rolling(window = '30D', closed = 'left').sum(), on = ['Cardnum','Merchnum','Date'], rsuffix = '_cn_mn_30d_sum')
new_df['Amount_cn_mn_30d_act-avg'] = new_df['Amount_cn_mn_actual'] - new_df['Amount_cn_mn_30d_avg']
new_df['Amount_cn_mn_30d_act/avg'] = new_df['Amount_cn_mn_actual']/new_df['Amount_cn_mn_30d_avg']
new_df['Amount_cn_mn_30d_act/sum'] = new_df['Amount_cn_mn_actual']/new_df['Amount_cn_mn_30d_sum']

In [281]:
df2 = df.groupby(['Cardnum','Merch zip','Date'])['Amount'].sum().to_frame().reset_index().set_index('Date')
new_df = new_df.join(df.groupby(['Cardnum','Merch zip','Date'])['Amount'].sum(),on = ['Cardnum','Merch zip','Date'], rsuffix = '_cn_zip_actual')

new_df = new_df.join(df2.groupby(['Cardnum','Merch zip'])['Amount'].rolling(window = '1D', closed = 'left').mean(), on = ['Cardnum','Merch zip','Date'], rsuffix = '_cn_zip_1d_avg')
new_df = new_df.join(df2.groupby(['Cardnum','Merch zip'])['Amount'].rolling(window = '1D', closed = 'left').sum(), on = ['Cardnum','Merch zip','Date'], rsuffix = '_cn_zip_1d_sum')
new_df['Amount_cn_zip_1d_act-avg'] = new_df['Amount_cn_zip_actual'] - new_df['Amount_cn_zip_1d_avg']
new_df['Amount_cn_zip_1d_act/avg'] = new_df['Amount_cn_zip_actual']/new_df['Amount_cn_zip_1d_avg']
new_df['Amount_cn_zip_1d_act/sum'] = new_df['Amount_cn_zip_actual']/new_df['Amount_cn_zip_1d_sum']

new_df = new_df.join(df2.groupby(['Cardnum','Merch zip'])['Amount'].rolling(window = '4D', closed = 'left').mean(), on = ['Cardnum','Merch zip','Date'], rsuffix = '_cn_zip_4d_avg')
new_df = new_df.join(df2.groupby(['Cardnum','Merch zip'])['Amount'].rolling(window = '4D', closed = 'left').sum(), on = ['Cardnum','Merch zip','Date'], rsuffix = '_cn_zip_4d_sum')
new_df['Amount_cn_zip_4d_act-avg'] = new_df['Amount_cn_zip_actual'] - new_df['Amount_cn_zip_4d_avg']
new_df['Amount_cn_zip_4d_act/avg'] = new_df['Amount_cn_zip_actual']/new_df['Amount_cn_zip_4d_avg']
new_df['Amount_cn_zip_4d_act/sum'] = new_df['Amount_cn_zip_actual']/new_df['Amount_cn_zip_4d_sum']

new_df = new_df.join(df2.groupby(['Cardnum','Merch zip'])['Amount'].rolling(window = '7D', closed = 'left').mean(), on = ['Cardnum','Merch zip','Date'], rsuffix = '_cn_zip_7d_avg')
new_df = new_df.join(df2.groupby(['Cardnum','Merch zip'])['Amount'].rolling(window = '7D', closed = 'left').sum(), on = ['Cardnum','Merch zip','Date'], rsuffix = '_cn_zip_7d_sum')
new_df['Amount_cn_zip_7d_act-avg'] = new_df['Amount_cn_zip_actual'] - new_df['Amount_cn_zip_7d_avg']
new_df['Amount_cn_zip_7d_act/avg'] = new_df['Amount_cn_zip_actual']/new_df['Amount_cn_zip_7d_avg']
new_df['Amount_cn_zip_7d_act/sum'] = new_df['Amount_cn_zip_actual']/new_df['Amount_cn_zip_7d_sum']

new_df = new_df.join(df2.groupby(['Cardnum','Merch zip'])['Amount'].rolling(window = '14D', closed = 'left').mean(), on = ['Cardnum','Merch zip','Date'], rsuffix = '_cn_zip_14d_avg')
new_df = new_df.join(df2.groupby(['Cardnum','Merch zip'])['Amount'].rolling(window = '14D', closed = 'left').sum(), on = ['Cardnum','Merch zip','Date'], rsuffix = '_cn_zip_14d_sum')
new_df['Amount_cn_zip_14d_act-avg'] = new_df['Amount_cn_zip_actual'] - new_df['Amount_cn_zip_14d_avg']
new_df['Amount_cn_zip_14d_act/avg'] = new_df['Amount_cn_zip_actual']/new_df['Amount_cn_zip_14d_avg']
new_df['Amount_cn_zip_14d_act/sum'] = new_df['Amount_cn_zip_actual']/new_df['Amount_cn_zip_14d_sum']

new_df = new_df.join(df2.groupby(['Cardnum','Merch zip'])['Amount'].rolling(window = '30D', closed = 'left').mean(), on = ['Cardnum','Merch zip','Date'], rsuffix = '_cn_zip_30d_avg')
new_df = new_df.join(df2.groupby(['Cardnum','Merch zip'])['Amount'].rolling(window = '30D', closed = 'left').sum(), on = ['Cardnum','Merch zip','Date'], rsuffix = '_cn_zip_30d_sum')
new_df['Amount_cn_zip_30d_act-avg'] = new_df['Amount_cn_zip_actual'] - new_df['Amount_cn_zip_30d_avg']
new_df['Amount_cn_zip_30d_act/avg'] = new_df['Amount_cn_zip_actual']/new_df['Amount_cn_zip_30d_avg']
new_df['Amount_cn_zip_30d_act/sum'] = new_df['Amount_cn_zip_actual']/new_df['Amount_cn_zip_30d_sum']

In [282]:
df2 = df.groupby(['Cardnum','Merch state','Date'])['Amount'].sum().to_frame().reset_index().set_index('Date')
new_df = new_df.join(df.groupby(['Cardnum','Merch state','Date'])['Amount'].sum(),on = ['Cardnum','Merch state','Date'], rsuffix = '_cn_st_actual')

new_df = new_df.join(df2.groupby(['Cardnum','Merch state'])['Amount'].rolling(window = '1D', closed = 'left').mean(), on = ['Cardnum','Merch state','Date'], rsuffix = '_cn_st_1d_avg')
new_df = new_df.join(df2.groupby(['Cardnum','Merch state'])['Amount'].rolling(window = '1D', closed = 'left').sum(), on = ['Cardnum','Merch state','Date'], rsuffix = '_cn_st_1d_sum')
new_df['Amount_cn_st_1d_act-avg'] = new_df['Amount_cn_st_actual'] - new_df['Amount_cn_st_1d_avg']
new_df['Amount_cn_st_1d_act/avg'] = new_df['Amount_cn_st_actual']/new_df['Amount_cn_st_1d_avg']
new_df['Amount_cn_st_1d_act/sum'] = new_df['Amount_cn_st_actual']/new_df['Amount_cn_st_1d_sum']

new_df = new_df.join(df2.groupby(['Cardnum','Merch state'])['Amount'].rolling(window = '4D', closed = 'left').mean(), on = ['Cardnum','Merch state','Date'], rsuffix = '_cn_st_4d_avg')
new_df = new_df.join(df2.groupby(['Cardnum','Merch state'])['Amount'].rolling(window = '4D', closed = 'left').sum(), on = ['Cardnum','Merch state','Date'], rsuffix = '_cn_st_4d_sum')
new_df['Amount_cn_st_4d_act-avg'] = new_df['Amount_cn_st_actual'] - new_df['Amount_cn_st_4d_avg']
new_df['Amount_cn_st_4d_act/avg'] = new_df['Amount_cn_st_actual']/new_df['Amount_cn_st_4d_avg']
new_df['Amount_cn_st_4d_act/sum'] = new_df['Amount_cn_st_actual']/new_df['Amount_cn_st_4d_sum']

new_df = new_df.join(df2.groupby(['Cardnum','Merch state'])['Amount'].rolling(window = '7D', closed = 'left').mean(), on = ['Cardnum','Merch state','Date'], rsuffix = '_cn_st_7d_avg')
new_df = new_df.join(df2.groupby(['Cardnum','Merch state'])['Amount'].rolling(window = '7D', closed = 'left').sum(), on = ['Cardnum','Merch state','Date'], rsuffix = '_cn_st_7d_sum')
new_df['Amount_cn_st_7d_act-avg'] = new_df['Amount_cn_st_actual'] - new_df['Amount_cn_st_7d_avg']
new_df['Amount_cn_st_7d_act/avg'] = new_df['Amount_cn_st_actual']/new_df['Amount_cn_st_7d_avg']
new_df['Amount_cn_st_7d_act/sum'] = new_df['Amount_cn_st_actual']/new_df['Amount_cn_st_7d_sum']

new_df = new_df.join(df2.groupby(['Cardnum','Merch state'])['Amount'].rolling(window = '14D', closed = 'left').mean(), on = ['Cardnum','Merch state','Date'], rsuffix = '_cn_st_14d_avg')
new_df = new_df.join(df2.groupby(['Cardnum','Merch state'])['Amount'].rolling(window = '14D', closed = 'left').sum(), on = ['Cardnum','Merch state','Date'], rsuffix = '_cn_st_14d_sum')
new_df['Amount_cn_st_14d_act-avg'] = new_df['Amount_cn_st_actual'] - new_df['Amount_cn_st_14d_avg']
new_df['Amount_cn_st_14d_act/avg'] = new_df['Amount_cn_st_actual']/new_df['Amount_cn_st_14d_avg']
new_df['Amount_cn_st_14d_act/sum'] = new_df['Amount_cn_st_actual']/new_df['Amount_cn_st_14d_sum']

new_df = new_df.join(df2.groupby(['Cardnum','Merch state'])['Amount'].rolling(window = '30D', closed = 'left').mean(), on = ['Cardnum','Merch state','Date'], rsuffix = '_cn_st_30d_avg')
new_df = new_df.join(df2.groupby(['Cardnum','Merch state'])['Amount'].rolling(window = '30D', closed = 'left').sum(), on = ['Cardnum','Merch state','Date'], rsuffix = '_cn_st_30d_sum')
new_df['Amount_cn_st_30d_act-avg'] = new_df['Amount_cn_st_actual'] - new_df['Amount_cn_st_30d_avg']
new_df['Amount_cn_st_30d_act/avg'] = new_df['Amount_cn_st_actual']/new_df['Amount_cn_st_30d_avg']
new_df['Amount_cn_st_30d_act/sum'] = new_df['Amount_cn_st_actual']/new_df['Amount_cn_st_30d_sum']

In [284]:
### frequency variables
new_df['Date'] = new_df['Date'].astype('datetime64[ns]')

# Cardnum
df3 = df.set_index('Date').groupby(['Cardnum'])['Recnum'].rolling(window = '1D', closed = 'left').count().to_frame().reset_index().drop_duplicates(subset = ['Cardnum','Date'],keep = 'first')
new_df = new_df.merge(df3.rename(columns = {'Recnum':'Freq_cn_1d'}), on = ['Cardnum','Date'])

df3 = df.set_index('Date').groupby(['Cardnum'])['Recnum'].rolling(window = '4D', closed = 'left').count().to_frame().reset_index().drop_duplicates(subset = ['Cardnum','Date'],keep = 'first')
new_df = new_df.merge(df3.rename(columns = {'Recnum':'Freq_cn_4d'}), on = ['Cardnum','Date'])

df3 = df.set_index('Date').groupby(['Cardnum'])['Recnum'].rolling(window = '7D', closed = 'left').count().to_frame().reset_index().drop_duplicates(subset = ['Cardnum','Date'],keep = 'first')
new_df = new_df.merge(df3.rename(columns = {'Recnum':'Freq_cn_7d'}), on = ['Cardnum','Date'])

df3 = df.set_index('Date').groupby(['Cardnum'])['Recnum'].rolling(window = '14D', closed = 'left').count().to_frame().reset_index().drop_duplicates(subset = ['Cardnum','Date'],keep = 'first')
new_df = new_df.merge(df3.rename(columns = {'Recnum':'Freq_cn_14d'}), on = ['Cardnum','Date'])

df3 = df.set_index('Date').groupby(['Cardnum'])['Recnum'].rolling(window = '30D', closed = 'left').count().to_frame().reset_index().drop_duplicates(subset = ['Cardnum','Date'],keep = 'first')
new_df = new_df.merge(df3.rename(columns = {'Recnum':'Freq_cn_30d'}), on = ['Cardnum','Date'])

# Merchnum
df3 = df.set_index('Date').groupby(['Merchnum'])['Recnum'].rolling(window = '1D', closed = 'left').count().to_frame().reset_index().drop_duplicates(subset = ['Merchnum','Date'],keep = 'first')
new_df = new_df.merge(df3.rename(columns = {'Recnum':'Freq_mn_1d'}), on = ['Merchnum','Date'])

df3 = df.set_index('Date').groupby(['Merchnum'])['Recnum'].rolling(window = '4D', closed = 'left').count().to_frame().reset_index().drop_duplicates(subset = ['Merchnum','Date'],keep = 'first')
new_df = new_df.merge(df3.rename(columns = {'Recnum':'Freq_mn_4d'}), on = ['Merchnum','Date'])

df3 = df.set_index('Date').groupby(['Merchnum'])['Recnum'].rolling(window = '7D', closed = 'left').count().to_frame().reset_index().drop_duplicates(subset = ['Merchnum','Date'],keep = 'first')
new_df = new_df.merge(df3.rename(columns = {'Recnum':'Freq_mn_7d'}), on = ['Merchnum','Date'])

df3 = df.set_index('Date').groupby(['Merchnum'])['Recnum'].rolling(window = '14D', closed = 'left').count().to_frame().reset_index().drop_duplicates(subset = ['Merchnum','Date'],keep = 'first')
new_df = new_df.merge(df3.rename(columns = {'Recnum':'Freq_mn_14d'}), on = ['Merchnum','Date'])

df3 = df.set_index('Date').groupby(['Merchnum'])['Recnum'].rolling(window = '30D', closed = 'left').count().to_frame().reset_index().drop_duplicates(subset = ['Merchnum','Date'],keep = 'first')
new_df = new_df.merge(df3.rename(columns = {'Recnum':'Freq_mn_30d'}), on = ['Merchnum','Date'])


In [285]:
# Cardnum & Merchnum
df3 = df.set_index('Date').groupby(['Cardnum','Merchnum'])['Recnum'].rolling(window = '1D', closed = 'left').count().to_frame().reset_index().drop_duplicates(subset = ['Cardnum','Merchnum','Date'],keep = 'first')
new_df = new_df.merge(df3.rename(columns = {'Recnum':'Freq_cn_mn_1d'}), on = ['Cardnum','Merchnum','Date'])

df3 = df.set_index('Date').groupby(['Cardnum','Merchnum'])['Recnum'].rolling(window = '4D', closed = 'left').count().to_frame().reset_index().drop_duplicates(subset = ['Cardnum','Merchnum','Date'],keep = 'first')
new_df = new_df.merge(df3.rename(columns = {'Recnum':'Freq_cn_mn_4d'}), on = ['Cardnum','Merchnum','Date'])

df3 = df.set_index('Date').groupby(['Cardnum','Merchnum'])['Recnum'].rolling(window = '7D', closed = 'left').count().to_frame().reset_index().drop_duplicates(subset = ['Cardnum','Merchnum','Date'],keep = 'first')
new_df = new_df.merge(df3.rename(columns = {'Recnum':'Freq_cn_mn_7d'}), on = ['Cardnum','Merchnum','Date'])

df3 = df.set_index('Date').groupby(['Cardnum','Merchnum'])['Recnum'].rolling(window = '14D', closed = 'left').count().to_frame().reset_index().drop_duplicates(subset = ['Cardnum','Merchnum','Date'],keep = 'first')
new_df = new_df.merge(df3.rename(columns = {'Recnum':'Freq_cn_mn_14d'}), on = ['Cardnum','Merchnum','Date'])

df3 = df.set_index('Date').groupby(['Cardnum','Merchnum'])['Recnum'].rolling(window = '30D', closed = 'left').count().to_frame().reset_index().drop_duplicates(subset = ['Cardnum','Merchnum','Date'],keep = 'first')
new_df = new_df.merge(df3.rename(columns = {'Recnum':'Freq_cn_mn_30d'}), on = ['Cardnum','Merchnum','Date'])

# Cardnum & Merch zip
df3 = df.set_index('Date').groupby(['Cardnum','Merch zip'])['Recnum'].rolling(window = '1D', closed = 'left').count().to_frame().reset_index().drop_duplicates(subset = ['Cardnum','Merch zip','Date'],keep = 'first')
new_df = new_df.merge(df3.rename(columns = {'Recnum':'Freq_cn_zip_1d'}), on = ['Cardnum','Merch zip','Date'])

df3 = df.set_index('Date').groupby(['Cardnum','Merch zip'])['Recnum'].rolling(window = '4D', closed = 'left').count().to_frame().reset_index().drop_duplicates(subset = ['Cardnum','Merch zip','Date'],keep = 'first')
new_df = new_df.merge(df3.rename(columns = {'Recnum':'Freq_cn_zip_4d'}), on = ['Cardnum','Merch zip','Date'])

df3 = df.set_index('Date').groupby(['Cardnum','Merch zip'])['Recnum'].rolling(window = '7D', closed = 'left').count().to_frame().reset_index().drop_duplicates(subset = ['Cardnum','Merch zip','Date'],keep = 'first')
new_df = new_df.merge(df3.rename(columns = {'Recnum':'Freq_cn_zip_7d'}), on = ['Cardnum','Merch zip','Date'])

df3 = df.set_index('Date').groupby(['Cardnum','Merch zip'])['Recnum'].rolling(window = '14D', closed = 'left').count().to_frame().reset_index().drop_duplicates(subset = ['Cardnum','Merch zip','Date'],keep = 'first')
new_df = new_df.merge(df3.rename(columns = {'Recnum':'Freq_cn_zip_14d'}), on = ['Cardnum','Merch zip','Date'])

df3 = df.set_index('Date').groupby(['Cardnum','Merch zip'])['Recnum'].rolling(window = '30D', closed = 'left').count().to_frame().reset_index().drop_duplicates(subset = ['Cardnum','Merch zip','Date'],keep = 'first')
new_df = new_df.merge(df3.rename(columns = {'Recnum':'Freq_cn_zip_30d'}), on = ['Cardnum','Merch zip','Date'])


In [286]:
# Cardnum & Merch state
df3 = df.set_index('Date').groupby(['Cardnum','Merch state'])['Recnum'].rolling(window = '1D', closed = 'left').count().to_frame().reset_index().drop_duplicates(subset = ['Cardnum','Merch state','Date'],keep = 'first')
new_df = new_df.merge(df3.rename(columns = {'Recnum':'Freq_cn_st_1d'}), on = ['Cardnum','Merch state','Date'])

df3 = df.set_index('Date').groupby(['Cardnum','Merch state'])['Recnum'].rolling(window = '4D', closed = 'left').count().to_frame().reset_index().drop_duplicates(subset = ['Cardnum','Merch state','Date'],keep = 'first')
new_df = new_df.merge(df3.rename(columns = {'Recnum':'Freq_cn_st_4d'}), on = ['Cardnum','Merch state','Date'])

df3 = df.set_index('Date').groupby(['Cardnum','Merch state'])['Recnum'].rolling(window = '7D', closed = 'left').count().to_frame().reset_index().drop_duplicates(subset = ['Cardnum','Merch state','Date'],keep = 'first')
new_df = new_df.merge(df3.rename(columns = {'Recnum':'Freq_cn_st_7d'}), on = ['Cardnum','Merch state','Date'])

df3 = df.set_index('Date').groupby(['Cardnum','Merch state'])['Recnum'].rolling(window = '14D', closed = 'left').count().to_frame().reset_index().drop_duplicates(subset = ['Cardnum','Merch state','Date'],keep = 'first')
new_df = new_df.merge(df3.rename(columns = {'Recnum':'Freq_cn_st_14d'}), on = ['Cardnum','Merch state','Date'])

df3 = df.set_index('Date').groupby(['Cardnum','Merch state'])['Recnum'].rolling(window = '30D', closed = 'left').count().to_frame().reset_index().drop_duplicates(subset = ['Cardnum','Merch state','Date'],keep = 'first')
new_df = new_df.merge(df3.rename(columns = {'Recnum':'Freq_cn_st_30d'}), on = ['Cardnum','Merch state','Date'])

In [287]:
len(list(new_df))

165

In [288]:
# Days since expert variables
df3 = df.set_index('Date').groupby(['Cardnum'])['Recnum'].rolling(window = '1D', closed = 'left').count().to_frame().reset_index().drop_duplicates(subset = ['Cardnum','Date'])
df3['Date_lag'] = df3.groupby('Cardnum')['Date'].shift(1)
df3['last_cn'] = (df3['Date'] - df3['Date_lag']).dt.days
new_df = new_df.merge(df3[['Cardnum','Date','last_cn']])

df3 = df.set_index('Date').groupby(['Merchnum'])['Recnum'].rolling(window = '1D', closed = 'left').count().to_frame().reset_index().drop_duplicates(subset = ['Merchnum','Date'])
df3['Date_lag'] = df3.groupby('Merchnum')['Date'].shift(1)
df3['last_mn'] = (df3['Date'] - df3['Date_lag']).dt.days
new_df = new_df.merge(df3[['Merchnum','Date','last_mn']])

df3 = df.set_index('Date').groupby(['Cardnum','Merchnum'])['Recnum'].rolling(window = '1D', closed = 'left').count().to_frame().reset_index().drop_duplicates(subset = ['Cardnum','Merchnum','Date'])
df3['Date_lag'] = df3.groupby(['Cardnum','Merchnum'])['Date'].shift(1)
df3['last_cn_mn'] = (df3['Date'] - df3['Date_lag']).dt.days
new_df = new_df.merge(df3[['Cardnum','Merchnum','Date','last_cn_mn']])

df3 = df.set_index('Date').groupby(['Cardnum','Merch zip'])['Recnum'].rolling(window = '1D', closed = 'left').count().to_frame().reset_index().drop_duplicates(subset = ['Cardnum','Merch zip','Date'])
df3['Date_lag'] = df3.groupby(['Cardnum','Merch zip'])['Date'].shift(1)
df3['last_cn_zip'] = (df3['Date'] - df3['Date_lag']).dt.days
new_df = new_df.merge(df3[['Cardnum','Merch zip','Date','last_cn_zip']])

df3 = df.set_index('Date').groupby(['Cardnum','Merch state'])['Recnum'].rolling(window = '1D', closed = 'left').count().to_frame().reset_index().drop_duplicates(subset = ['Cardnum','Merch state','Date'])
df3['Date_lag'] = df3.groupby(['Cardnum','Merch state'])['Date'].shift(1)
df3['last_cn_st'] = (df3['Date'] - df3['Date_lag']).dt.days
new_df = new_df.merge(df3[['Cardnum','Merch state','Date','last_cn_st']])


In [289]:
list(new_df)

['Recnum',
 'Cardnum',
 'Date',
 'Merchnum',
 'Merch description',
 'Merch state',
 'Merch zip',
 'Transtype',
 'Amount',
 'Fraud',
 'Amount_cn_actual',
 'Amount_cn_1d_avg',
 'Amount_cn_1d_sum',
 'Amount_cn_1d_act-avg',
 'Amount_cn_1d_act/avg',
 'Amount_cn_1d_act/sum',
 'Amount_cn_4d_avg',
 'Amount_cn_4d_sum',
 'Amount_cn_4d_act-avg',
 'Amount_cn_4d_act/avg',
 'Amount_cn_4d_act/sum',
 'Amount_cn_7d_avg',
 'Amount_cn_7d_sum',
 'Amount_cn_7d_act-avg',
 'Amount_cn_7d_act/avg',
 'Amount_cn_7d_act/sum',
 'Amount_cn_14d_avg',
 'Amount_cn_14d_sum',
 'Amount_cn_14d_act-avg',
 'Amount_cn_14d_act/avg',
 'Amount_cn_14d_act/sum',
 'Amount_cn_30d_avg',
 'Amount_cn_30d_sum',
 'Amount_cn_30d_act-avg',
 'Amount_cn_30d_act/avg',
 'Amount_cn_30d_act/sum',
 'Amount_mn_actual',
 'Amount_mn_1d_avg',
 'Amount_mn_1d_sum',
 'Amount_mn_1d_act-avg',
 'Amount_mn_1d_act/avg',
 'Amount_mn_1d_act/sum',
 'Amount_mn_4d_avg',
 'Amount_mn_4d_sum',
 'Amount_mn_4d_act-avg',
 'Amount_mn_4d_act/avg',
 'Amount_mn_4d_act/sum

In [290]:
# Velocity deviation
new_df['velo_cn_avg7d'] = new_df['Freq_cn_1d']/(new_df['Freq_cn_7d']/7)
new_df['velo_cn_avg14d'] = new_df['Freq_cn_1d']/(new_df['Freq_cn_14d']/14)
new_df['velo_cn_avg30d'] = new_df['Freq_cn_1d']/(new_df['Freq_cn_30d']/30)
new_df['velo_mn_avg7d'] = new_df['Freq_mn_1d']/(new_df['Freq_mn_7d']/7)
new_df['velo_mn_avg14d'] = new_df['Freq_mn_1d']/(new_df['Freq_mn_14d']/14)
new_df['velo_mn_avg30d'] = new_df['Freq_mn_1d']/(new_df['Freq_mn_30d']/30)




In [294]:
# fill na with 0
new_df = new_df.fillna(0)

# export amount expert variables
new_df.to_csv('AmountVariables.csv')

In [296]:
new_df.tail()
len(list(new_df))

176

In [ ]:
# new_df = pd.read_csv('AmountVariables.csv')
